In [1]:
input_repr_dict = catalog.load('input_repr_dict')

[07/05/24 15:09:44] INFO     Loading data from input_repr_dict (PickleDataset)...               ]8;id=607662;file://C:\Users\omid\miniconda3\envs\EIT-Epsilon\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=276363;file://C:\Users\omid\miniconda3\envs\EIT-Epsilon\Lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

In [2]:
input_repr_dict


{
    'J': [
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 4, 5, 6, 7],
        [1, 2, 3, 6, 7],
        [1, 2, 3, 6, 7],
        [1, 2, 3, 6, 7],
        [1, 2, 3, 6, 7],
        [1, 2, 3, 6, 7],
        [1, 2, 3, 6, 7]


In [3]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import math
import re
import random
import logging
from pandas.api.types import is_string_dtype
import plotly
from typing import List, Dict, Tuple, Union

# Instantiate logger
logger = logging.getLogger(__name__)


class Job:
    """
    The Job class contains methods for preprocessing and extracting information from open orders that need
    to be processed in a manufacturing workshop.
    """

    @staticmethod
    def filter_in_scope_op1(data: pd.DataFrame) -> pd.DataFrame:
        """
        Filters the data to include only in-scope operations for OP 1.

        Args:
            data (pd.DataFrame): The input data.

        Returns:
            pd.DataFrame: The filtered data.
        """
        # Debug statement
        logger.info(f"Total order data: {data.shape}")

        # Apply the filter
        inscope_data = data[
            (
                data["Part Description"].str.contains("OP 1")
                | data["Part Description"].str.contains("ATT ")
            )
            & (~data["On Hold?"])
            & (~data["Part Description"].str.contains("OP 2"))
        ]

        # Debug statement
        logger.info(f"In-scope data for OP 1: {inscope_data.shape}")

        return inscope_data

    @staticmethod
    def extract_info(data: pd.DataFrame) -> pd.DataFrame:
        """
        Extracts type, size, and orientation from the part description.

        Args:
            data (pd.DataFrame): The input data.

        Returns:
            pd.DataFrame: The data with extracted information.
        """
        data = data.assign(
            Type=lambda x: x["Part Description"].apply(
                lambda y: "CR" if "CR" in y else ("PS" if "PS" in y else "")
            ),
            Size=lambda x: x["Part Description"].apply(
                lambda y: (re.search(r"Sz (\d+N?)", y).group(1) if re.search(r"Sz (\d+N?)", y) else "")
            ),
            Orientation=lambda x: x["Part Description"].apply(
                lambda y: ("LEFT" if "LEFT" in y.upper() else ("RIGHT" if "RIGHT" in y.upper() else ""))
            ),
            Cementless=lambda x: x["Part Description"].apply(
                lambda y: "CLS" if "CLS" in y.upper() else "C"
            ),
        )

        # Debug statement
        if data[["Type", "Size", "Orientation"]].isna().sum().sum() > 0:
            logger.warning(
                f"Data with extracted information: {data[['Type', 'Size', 'Orientation']].isna().sum()}"
            )
        else:
            logger.info(f"No missing values in Type, Size, and Orientation columns")

        return data

    @staticmethod
    def check_part_id_consistency(data: pd.DataFrame) -> None:
        """
        Checks the consistency of Part IDs.

        Args:
            data (pd.DataFrame): The input data.

        Raises:
            LoggerError: If Part ID is not unique for every combination of Type, Size, and Orientation.
        """
        grouped = data.groupby("Part ID")[["Type", "Size", "Orientation"]].nunique()

        if (grouped > 1).any().any():
            logger.error(
                "[bold red blink]Part ID not unique for every combination of Type, Size, and Orientation[/]",
                extra={"markup": True},
            )
        else:
            logger.info(f"Part ID consistency check passed")

    @staticmethod
    def create_jobs_op1(data: pd.DataFrame) -> List[List[int]]:
        """
        Creates jobs representation for the GA, defined as J.
        Cemented products do not have to go through manual prep in OP 1.

        Args:
            data (pd.DataFrame): The input data.

        Returns:
            List[List[int]]: The list of jobs.
        """
        # Find proportion of cementless products
        cementless_count = data[data["Cementless"] == "CLS"].shape[0]
        total_products = data.shape[0]

        cementless_percentage = (cementless_count / total_products) * 100
        logger.info(f"Proportion of cementless products: {cementless_percentage:.1f}%")

        # Populate J
        J = [
            [1, 2, 3, 4, 5, 6, 7] if cementless == "CLS" else [1, 2, 3, 6, 7]
            for cementless in data["Cementless"]
        ]

        if not len(J) == len(data):
            logger.error(
                "[bold red blink]J is not of the same length as processed orders![/]",
                extra={"markup": True},
            )

        # Debug statement
        logger.info(f"Snippet of Jobs for OP 1: {J[:2]}")

        return J

    @staticmethod
    def get_part_id(data: pd.DataFrame) -> List[int]:
        part_id = data["Part ID"].to_list()

        # Show snippet of Part IDs
        logger.info(f"Snippet of Part IDs: {part_id[:5]}")

        return part_id


class Shop:
    """
    The Shop class contains methods for creating machine lists, compatibility matrices,
    duration matrices and due dates as input for a genetic algorithm.
    It creates a digital representation of the processes in and the setup of a manufacturing workshop.
    """

    @staticmethod
    def create_machines(machine_qty_dict: Dict[str, int]) -> List[int]:
        """
        Creates a list of machines based on the quantity dictionary.

        Args:
            machine_qty_dict (Dict[str, int]): The dictionary of machine quantities.

        Returns:
            List[int]: The list of machines.
        """
        total_machines = sum(machine_qty_dict.values())
        M = list(range(1, total_machines + 1))

        # Debug statement
        logger.info(f"Machine list (M): {M}")

        return M

    @staticmethod
    def get_compatibility(
        J: List[List[int]], task_to_machines: Dict[int, List[int]]
    ) -> List[List[List[int]]]:
        """
        Gets the compatibility of tasks to machines.

        Example:
            compat = [[[1], [2, 3], [4, 5]],  -- Job 1, task 1 is only compatible with machine 1
                    [[1, 2],[2, 3],[4, 5]]]  -- Job 2, task 1 is compatible with machine 1 and 2

        Args:
            J (List[List[int]]): The list of jobs.
            task_to_machines (Dict[int, List[int]]): The dictionary mapping tasks to machines.

        Returns:
            List[List[List[int]]]: The compatibility list.
        """
        compat = []
        for job_tasks in J:
            job_compat = []
            for task in job_tasks:
                if task in task_to_machines:
                    machines = task_to_machines[task]
                else:
                    raise ValueError("Invalid task number!")
                job_compat.append(machines)
            compat.append(job_compat)

        # Debug statement
        logger.info(f"Snippet of compatability matrix: {compat[0]}")

        return compat

    @staticmethod
    def preprocess_cycle_times(
        cycle_times: pd.DataFrame, last_task_minutes: int = 4
    ) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Preprocesses the cycle times:
            1.) Remove empty rows and columns
            2.) Create new index starting from 1
            3.) Reduce column names to only the size
            4.) Fill the missing values in final inspection with 4 minutes
            5.) Split data for cruciate retaining and posterior stabilizing products

        Args:
            cycle_times (pd.DataFrame): The cycle times data.
            last_task_minutes (int, optional): The duration of the last task. Defaults to 4 minutes.

        Returns:
            Tuple[pd.DataFrame, pd.DataFrame]: The preprocessed PS and CR cycle times.
        """
        cycle_times.columns = cycle_times.iloc[1]
        cycle_times = cycle_times.iloc[2:, 1:]
        cycle_times.index = range(1, len(cycle_times) + 1)

        def extract_number(s: str) -> str:
            match = re.search(r"\d+N?", s)
            return match.group(0) if match else s

        cycle_times.columns = [extract_number(col) for col in cycle_times.columns]
        cycle_times.loc[7] = cycle_times.loc[7].fillna(last_task_minutes)
        ps_times, cr_times = (
            cycle_times.iloc[:8, 2 : math.ceil(cycle_times.shape[1] / 2) + 1],
            cycle_times.iloc[:8, math.ceil(cycle_times.shape[1] / 2) + 1 :],
        )

        # Debug statement
        logger.info(f"PS times dim.: {ps_times.shape}, CR times dim.: {cr_times.shape}")

        # Convert everything to float
        ps_times = ps_times.astype(float)
        cr_times = cr_times.astype(float)

        # Check if all cycle times are numeric values
        if not all(isinstance(i, (int, float)) for i in ps_times.values.flatten()) or not all(
            isinstance(i, (int, float)) for i in cr_times.values.flatten()
        ):
            logger.warning(
                "[bold red blink]All cycle times should be numeric values. Please check the input data.[/]",
                extra={"markup": True},
            )

        # Define required sizes
        required_sizes = {"1", "2", "3", "3N", "4", "4N", "5", "5N", "6", "6N", "7", "8", "9", "10"}

        # Check if all required sizes are in the columns of both dataframes
        if not required_sizes.issubset(set(cr_times.columns)) or not required_sizes.issubset(
            set(ps_times.columns)
        ):
            logger.warning(
                "[bold red blink]Either cr_times or ps_times is missing some of the sizes in the columns.[/]",
                extra={"markup": True},
            )

        return ps_times, cr_times

    @staticmethod
    def get_duration_matrix(
        J: List[List[int]],
        inscope_orders: pd.DataFrame,
        cr_times: pd.DataFrame,
        ps_times: pd.DataFrame,
    ) -> List[List[float]]:
        """
        Gets the duration matrix for the jobs.

        Example:
             dur = [[3, 2, 2],  -- Job 1; task 1, 2, 3 will take 3, 2, 2 minutes respectively
                    [3, 3, 3]]  -- Job 2; task 1, 2, 3 will take 3, 3, 3 minutes respectively

        Args:
            J (List[List[int]]): The list of jobs.
            inscope_orders (pd.DataFrame): The in-scope orders.
            cr_times (pd.DataFrame): The CR cycle times.
            ps_times (pd.DataFrame): The PS cycle times.

        Returns:
            List[List[float]]: The duration matrix.
        """
        dur = []
        for i, job in enumerate(J):
            job_dur = []
            for task in job:
                times = cr_times if inscope_orders.iloc[i]["Type"] == "CR" else ps_times
                duration = round(
                    times.loc[task, inscope_orders.iloc[i]["Size"]]
                    * inscope_orders.iloc[i]["Order Qty"],
                    1,
                )
                job_dur.append(duration)
            dur.append(job_dur)

        # Debug statement
        logger.info(f"Snippet of duration: {dur[0]}")

        return dur

    @staticmethod
    def get_due_date(
        inscope_orders: pd.DataFrame,
        date: str = "2024-03-18",
        working_minutes: int = 480,
    ) -> List[int]:
        """
        Gets the due dates for the in-scope orders.

        Args:
            inscope_orders (pd.DataFrame): The in-scope orders.
            date (str): The reference date in 'YYYY-MM-DD' format. Defaults to '2024-03-18'.
            working_minutes (int, optional): The number of working minutes per day. Defaults to 480.

        Returns:
            List[int]: The list of due dates in working minutes.
        """
        due = []
        for due_date in inscope_orders["Due Date "]:
            if pd.Timestamp(date) > due_date:
                working_days = -len(pd.bdate_range(due_date, date)) * working_minutes
            else:
                working_days = len(pd.bdate_range(date, due_date)) * working_minutes
            due.append(working_days)

        # Debug statement
        logger.info(f"Snippet of due: {due[:5]}")

        return due


class JobShop(Job, Shop):
    """
    The JobShop class combines the functionality of the Job and Shop classes.
    The Job class is used to preprocess orders into the correct representation for a genetic algorithm.
    The Shop class is used to create a representation of the factory floor: the machines and the constraints.

    By using the JobShop class all the functionality of the Job and Shop classes can be accessed.
    """

    def preprocess_orders(self, croom_open_orders: pd.DataFrame) -> pd.DataFrame:
        """
        Preprocesses the open orders.

        Args:
            croom_open_orders (pd.DataFrame): The open orders.

        Returns:
            pd.DataFrame: The preprocessed orders.
        """
        inscope_data = self.filter_in_scope_op1(croom_open_orders).pipe(self.extract_info)
        self.check_part_id_consistency(inscope_data)

        return inscope_data

    def build_ga_representation(
        self,
        croom_processed_orders: pd.DataFrame,
        cr_cycle_times: pd.DataFrame,
        ps_cycle_times: pd.DataFrame,
        machine_qty_dict: Dict[str, int],
        task_to_machines: Dict[int, List[int]],
    ) -> Dict[str, any]:
        """
        Builds the GA representation:
            J: List[List[int]]: The list of jobs, each job is a list of tasks.
            M: List[int]: The list of machines.
            compat: List[List[List[int]]]: The compatibility list.
            dur: List[List[float]]: The duration matrix.
            due: List[int]: The list of due dates in working minutes.

        Additionally, checks if all output variables are valid; they must be (nested) lists of integers/floats.

        Args:
            croom_processed_orders (pd.DataFrame): The processed orders.
            cr_cycle_times (pd.DataFrame): The CR cycle times.
            ps_cycle_times (pd.DataFrame): The PS cycle times.
            machine_qty_dict (Dict[str, int]): The machine quantity dictionary.
            task_to_machines (Dict[int, List[int]]): The task to machines dictionary.

        Returns:
            Dict[str, any]: The GA representation.
        """
        J = self.create_jobs_op1(croom_processed_orders)
        M = self.create_machines(machine_qty_dict)
        compat = self.get_compatibility(J, task_to_machines)
        dur = self.get_duration_matrix(J, croom_processed_orders, cr_cycle_times, ps_cycle_times)
        due = self.get_due_date(croom_processed_orders)
        part_id = self.get_part_id(croom_processed_orders)

        def is_nested_list_of_numbers(lst):
            if isinstance(lst, list):
                return all(
                    (
                        is_nested_list_of_numbers(item)
                        if isinstance(item, list)
                        else isinstance(item, (int, float))
                    )
                    for item in lst
                )
            return False

        input_repr_dict = {
            "J": J,
            "M": M,
            "compat": compat,
            "dur": dur,
            "due": due,
            "part_id": part_id,
        }

        # Check if J, M, compat, dur, and due are (nested) lists of integers or floats
        # Part ID is a list of strings, so should be excluded
        for var_name, var in input_repr_dict.items():
            if var_name != "part_id":
                if not is_nested_list_of_numbers(var):
                    logger.error(
                        f"[bold red blink]{var_name} is not a nested list of integers/floats: {var[0]}[/]",
                        extra={"markup": True},
                    )

        return input_repr_dict


class GeneticAlgorithmScheduler:
    """
    Contains all functions to run a genetic algorithm for a flexible job shop scheduling problem (FJSSP):

    - Initialize population
    - Evaluate fitness
    - Crossover/Offspring
    - Mutation (Currently not implemented)
    - Run
    """

    def __init__(self):
        self.J = None
        self.M = None
        self.compat = None
        self.dur = None
        self.due = None
        self.part_id = None
        self.n = None
        self.P = None
        self.scores = None
        self.day_range = None
        self.best_schedule = None
        self.minutes_per_day = None
        self.change_over_time = None

    def find_avail_m(self, start: int, job_idx: int, task_num: int) -> int:
        """
        Finds the next available time for a machine to start a task, considering the working day duration.

        Args:
            start (int): The starting time in the schedule in minutes.
            job_idx (int): The index of the job in the job list.
            task_num (int): The task number within the job.

        Returns:
            int: The next available time for the machine to start the task.
        """
        for day in self.day_range:
            if start < day <= start + self.dur[job_idx][task_num]:
                return day
        return start + self.dur[job_idx][task_num]

    def init_population(self, num_inds: int = None, fill_inds: bool = False):
        """
        Initializes the population of schedules. Each schedule is a list of tasks assigned to machines with start times.
        """
        if num_inds is None:
            num_inds = self.n

        P = []
        percentages = np.arange(10, 101, 10)

        for i in range(num_inds):
            avail_m = {m: 0 for m in self.M}
            product_m = {m: 0 for m in self.M}
            changeover_finish_time = 0
            P_j = []

            J_temp = list(range(len(self.J)))
            random_roll = random.random()

            if random_roll < 0.5:
                random.shuffle(J_temp)
            # Larger chance to sort by part ID
            elif random_roll < 0.7:
                J_temp.sort(key=lambda x: self.part_id[x])
            # Larger chance to sort by due date
            elif random_roll < 0.9:
                J_temp.sort(key=lambda x: self.due[x], reverse=True)
            # Larger chance to sort by part ID and due date
            else:
                J_temp.sort(key=lambda x: (self.part_id[x], self.due[x]), reverse=True)

            while J_temp:
                # Pick the last item in the job array list
                job_idx = J_temp.pop()
                job = self.J[job_idx]
                for task in range(len(job)):
                    random_roll = random.random()

                    if task == 0:
                        compat_task_0 = self.compat[job_idx][task]
                        # Preferred machines: compatible machines for the task if they processed the same Part ID
                        # previously
                        preferred_machines = [
                            key
                            for key in compat_task_0
                            if product_m.get(key) == self.part_id[job_idx] or product_m.get(key) == 0
                        ]

                        if not preferred_machines:
                            m = (
                                min(compat_task_0, key=lambda x: avail_m.get(x))
                                # If no preferred machines, 80% chance to pick the one that comes available first
                                # among compatible machines, otherwise randomly pick a compatible machine
                                if random_roll < 0.8
                                else random.choice(compat_task_0)
                            )
                        else:
                            m = (
                                min(preferred_machines, key=lambda x: avail_m.get(x))
                                # If there are preferred machines, 90% chance to pick the one that comes available
                                # first among preferred machines, otherwise randomly pick a preferred machine
                                if random_roll < 0.9
                                else random.choice(preferred_machines)
                            )

                        # Start time for first task (HAAS machine) is defined as follows:
                        #  - If no changeover is required, start time is end time of previous task on that machine
                        #  - If a changeover is required, start time is end time of previous task plus the changeover
                        #    time, plus optional waiting time for changeover to finish if a changeover was already
                        #    happening on a different machine
                        start = (
                            avail_m[m]
                            if product_m[m] == 0 or self.part_id[job_idx] == product_m[m]
                            else avail_m[m]
                            + self.change_over_time
                            + max((changeover_finish_time - avail_m[m]), 0)
                        )

                        # Update changeover finish time
                        if product_m[m] != 0 and self.part_id[job_idx] != product_m[m]:
                            changeover_finish_time = start
                    else:
                        compat_with_task = self.compat[job_idx][task]
                        m = (
                            min(compat_with_task, key=lambda x: avail_m.get(x))
                            # Other tasks than milling & grinding on HAAS:
                            #  - 85% chance to pick the one that comes available first among compatible machines,
                            #    otherwise randomly pick a compatible machine
                            if random_roll < 0.85
                            else random.choice(compat_with_task)
                        )
                        start = max(avail_m[m], P_j[-1][3] + self.dur[job_idx][task - 1])

                    P_j.append((job_idx, job[task], m, start, self.dur[job_idx][task], task))
                    avail_m[m] = self.find_avail_m(start, job_idx, task)
                    product_m[m] = self.part_id[job_idx]

            if P_j not in P:
                P.append(P_j)
            else:
                i -= 1

            if not fill_inds and i * 100 / num_inds in percentages:
                logger.info(
                    f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {i * 100 / num_inds}% of schedules have been created."
                )

        if not fill_inds:
            self.P = P
        else:
            return P

    def evaluate_population(
        self,
        best_scores: list = None,
        display_scores: bool = True,
        on_time_bonus: int = 5000,
    ):
        """
        Evaluates the population of schedules by calculating a score for each schedule based on the completion times
        of jobs vs. the required due date.
        """
        self.scores = [
            round(
                sum(
                    (
                        self.due[job_idx]
                        - (start_time + self.dur[job_idx][-1])
                        + (
                            on_time_bonus
                            if (self.due[job_idx] - (start_time + self.dur[job_idx][-1])) > 0
                            else 0
                        )
                    )
                    for (
                        job_idx,
                        task,
                        machine,
                        start_time,
                        job_task_dur,
                        _,
                    ) in schedule
                    if task + 1 == max(self.J[job_idx])
                )
            )
            for schedule in self.P
        ]

        if display_scores:
            logger.info(
                f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Best score: {max(self.scores)}, "
                f"Median score: {np.median(self.scores)}, Worst score: {min(self.scores)}"
            )
            best_scores.append(max(self.scores))

    def resolve_conflict(
        self, P_prime: List[Tuple[int, int, int, int, int, int]]
    ) -> (List)[Tuple[int, int, int, int, int, int]]:
        """
        This function resolves conflicts in a given schedule. If tasks are planned on the same machine at the same time,
        it finds the first available time for each task to start on the machine.

        Parameters:
        P_prime (List[Tuple[int, int, int, int, int, int]]): A list of tuples where each tuple represents a task.
        Each task is represented as (job index, task, machine, start time, duration, task number).

        Returns:
        P_prime_sorted (List[Tuple[int, int, int, int, int, int]]): A sorted list of tuples where each tuple
        represents a task. Each task is represented as (job index, job, machine, start time, duration, task number).
        """
        P_prime_sorted = []
        avail_m = {m: 0 for m in self.M}
        product_m = {m: 0 for m in self.M}
        changeover_finish_time = 0
        start_times = {j: 0 for j in range(len(self.J))}

        for job_idx in range(len(self.J)):
            job = self.J[job_idx]
            job_tasks = sorted([entry for entry in P_prime if entry[0] == job_idx], key=lambda x: x[1])

            for task_entry in job_tasks:
                _, task, m, _, part_id, task_num = task_entry

                if task_num == 0:
                    start = (
                        avail_m[m]
                        if product_m[m] == 0 or self.part_id[job_idx] == product_m[m]
                        else avail_m[m]
                        + self.change_over_time
                        + max((changeover_finish_time - avail_m[m]), 0)
                    )
                    if product_m[m] != 0 and self.part_id[job_idx] != product_m[m]:
                        changeover_finish_time = start
                else:
                    start = max(
                        avail_m[m],
                        start_times[job_idx] + self.dur[job_idx][task_num - 1],
                    )
                start_times[job_idx] = start

                avail_m[m] = self.find_avail_m(start, job_idx, task_num)
                product_m[m] = self.part_id[job_idx]

                P_prime_sorted.append(
                    (
                        job_idx,
                        job[task_num],
                        m,
                        start,
                        self.dur[job_idx][task_num],
                        task_num,
                    )
                )

        return P_prime_sorted

    def offspring(self, n_e: float, n_c: float) -> None:
        """
        This function generates offspring for the next generation of the population. It identifies the best schedules
        in the population, after which it randomly selects each job from the first or the second schedule.
        Conflicts are resolved by another function.

        Parameters:
        n_e (float): The elitism rate. It represents the proportion of the population to be retained for the next generation.
        n_c (float): The crossover rate. It represents the proportion of the population to be generated through crossover.

        Returns:
        None. The function updates the population in-place.
        """
        self.evaluate_population(display_scores=False)
        scored_population = sorted(zip(self.scores, self.P), key=lambda x: x[0], reverse=True)
        retain_count = min(2, int(len(self.P) * n_e))
        P_0 = [schedule for score, schedule in scored_population[:retain_count]]

        iter_count = len(self.P) * (n_c + n_e)
        while len(P_0) < iter_count:
            P1, P2 = random.sample(P_0, 2)
            P_prime = [
                entry
                for job_idx in range(len(self.J))
                for entry in random.choice([P1, P2])
                if entry[0] == job_idx
            ]
            P_prime = self.resolve_conflict(P_prime)
            if P_prime not in P_0:
                P_0.append(P_prime)

        self.P = P_0

    def mutate(self):
        """
        WARNING: This function is currently not implemented and is a work in progress (WIP).

        Mutates the top 10% highest scoring schedules in the population by randomly picking new machines for the
        latest job. Does not seem to improve performance.
        """
        num_best_schedules = int(len(self.P) * 0.1)
        self.evaluate_population(display_scores=False)
        scored_population = sorted(zip(self.scores, self.P), key=lambda x: x[0], reverse=True)
        best_schedules = [schedule for score, schedule in scored_population[:num_best_schedules]]

        for schedule in best_schedules:
            sorted_schedule = sorted(schedule, key=lambda x: x[-3], reverse=True)
            unique_job_indices = list({entry[0] for entry in sorted_schedule[:4]})
            sorted_schedule_without_last_task = [
                t for t in sorted_schedule if t[0] not in unique_job_indices
            ]

            for job_idx in unique_job_indices:
                job = self.J[job_idx]
                for task in range(len(job)):
                    m = random.choice(self.compat[job_idx][task])
                    prev_task = next((t for t in sorted_schedule if t[2] == m), None)
                    if prev_task:
                        start = (
                            prev_task[3]
                            + prev_task[4]
                            + (
                                self.change_over_time
                                if task == 0 and self.part_id[prev_task[0]] != self.part_id[job_idx]
                                else 0
                            )
                        )
                    else:
                        start = 0
                    sorted_schedule_without_last_task.append(
                        (job_idx, job[task], m, start, self.dur[job_idx][task], task)
                    )

                test_scores = [
                    round(
                        sum(
                            (
                                self.due[job_idx]
                                - (start_time + self.dur[job_idx][-1])
                                + (
                                    5000
                                    if (self.due[job_idx] - (start_time + self.dur[job_idx][-1])) > 0
                                    else 0
                                )
                            )
                            for (
                                job_idx,
                                task,
                                machine,
                                start_time,
                                job_task_dur,
                                _,
                            ) in sched
                            if task + 1 == max(self.J[job_idx])
                        )
                    )
                    for sched in [schedule, sorted_schedule_without_last_task]
                ]

                if test_scores[1] > test_scores[0] and schedule in self.P:
                    self.P.remove(schedule)
                    self.P.append(sorted_schedule_without_last_task)

    def run(
        self,
        input_repr_dict: Dict[str, any],
        scheduling_options: dict,
        n: int = 1500,
        n_e: float = 0.1,
        n_c: float = 0.3,
        minutes_per_day: int = 480,
        max_iterations: int = 100,
    ):
        """
        Runs the genetic algorithm by initializing the population, evaluating it, and selecting the best schedule.

        Args:
            input_repr_dict (Dict[str, any]): A dictionary containing the necessary input variables for the GA.
            scheduling_options (dict): Dictionary containing the changeover time.
            n (int, optional): The population size. Defaults to 1200.
            n_e (float, optional): The elitism rate. Defaults to 0.1.
            n_c (float, optional): The crossover rate. Defaults to 0.3.
            minutes_per_day (int, optional): The number of working minutes per day. Defaults to 480.
            max_iterations (int, optional): The maximum number of iterations for the genetic algorithm. Defaults to 100.

        Returns:
            Tuple[List[Tuple[int, int, int, int, float]], List[int]]: The best schedule with the highest score and the list of best scores per generation.
        """
        self.J = input_repr_dict["J"]
        self.M = input_repr_dict["M"]
        self.compat = input_repr_dict["compat"]
        self.dur = input_repr_dict["dur"]
        self.due = input_repr_dict["due"]
        self.part_id = input_repr_dict["part_id"]
        self.n = n
        self.minutes_per_day = minutes_per_day
        self.change_over_time = scheduling_options["change_over_time"]
        self.day_range = np.arange(
            self.minutes_per_day,
            len(self.J) // 5 * self.minutes_per_day,
            self.minutes_per_day,
        )

        self.init_population()
        best_scores = []

        for iteration in range(max_iterations):
            logger.info(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Iteration {iteration + 1}")
            self.offspring(n_e=n_e, n_c=n_c)
            if len(self.P) < self.n:
                self.P += self.init_population(num_inds=self.n - len(self.P), fill_inds=True)
            self.evaluate_population(best_scores=best_scores)

        schedules_and_scores = sorted(zip(self.P, self.scores), key=lambda x: x[1], reverse=True)
        self.best_schedule = schedules_and_scores[0][0]
        logger.info(
            f"Snippet of best schedule (job, task, machine, start_time, duration): {self.best_schedule[:4]}"
        )

        return self.best_schedule, best_scores


def reformat_output(
    croom_processed_orders: pd.DataFrame,
    best_schedule: Dict[str, any],
    column_mapping_reformat: dict,
    machine_dict: dict,
) -> pd.DataFrame:
    """
    Reformats the output of the genetic algorithm by converting the best schedule into a dataframe,
    rounding the starting time, resetting and dropping the index, joining the best schedule to processed orders,
    defining the end time, renaming columns, applying machine name mapping, and creating start and end datetime
    based on a hypothetical start date.

    Args:
        croom_processed_orders (pd.DataFrame): The processed orders.
        best_schedule (Dict[str, any]): The best schedule generated by the genetic algorithm.
        column_mapping_reformat (dict): The mapping for renaming columns in the output dataframe.
        machine_dict (dict): The dictionary mapping machine numbers to machine names.

    Returns:
        pd.DataFrame: The reformatted output dataframe.
    """
    # Convert best schedule into a dataframe
    schedule_df = pd.DataFrame(
        best_schedule,
        columns=["job", "task", "machine", "starting_time", "duration", "task_num"],
    )

    # Round the starting time and duration
    schedule_df["starting_time"] = schedule_df["starting_time"].round(5)
    schedule_df["duration"] = schedule_df["duration"].round(5)

    # Reset and drop index
    croom_processed_orders.reset_index(inplace=True, drop=True)

    # Join best schedule to processed orders
    croom_processed_orders = croom_processed_orders.merge(
        schedule_df, left_index=True, right_on="job", how="left"
    )

    # Define end time
    croom_processed_orders["end_time"] = (
        croom_processed_orders["starting_time"] + croom_processed_orders["duration"]
    )

    # Rename columns
    croom_processed_orders = croom_processed_orders.rename(columns=column_mapping_reformat)

    # Apply machine name mapping
    croom_processed_orders["Machine"] = croom_processed_orders["Machine"].map(machine_dict)

    return croom_processed_orders


def create_start_end_time(
    croom_reformatted_orders: pd.DataFrame, scheduling_options: dict
) -> pd.DataFrame:
    """
    Adjusts the start and end times of tasks in the given DataFrame to fit within working hours (09:00 to 17:00)
    and ensures that tasks are scheduled sequentially within each job and machine.

    The function first converts the 'Start_time' from minutes to a datetime based on a hypothetical start date.
    It then adjusts the start and end times of tasks to fit within working hours, pushing tasks to the next day
    if they start after 17:00 or before 09:00. The function also checks for consistency in task scheduling within
    each job and machine.

    Args:
        croom_reformatted_orders (pd.DataFrame): The DataFrame containing reformatted orders with 'Start_time' in minutes.
        scheduling_options (dict): A dictionary containing scheduling options, including the 'start_date'.

    Returns:
        pd.DataFrame: The adjusted DataFrame with updated start and end times.
    """

    # Parse the date string into a datetime object
    base_date = datetime.strptime(scheduling_options["start_date"], "%Y-%m-%dT%H:%M")

    # Sort by start time ascending
    croom_reformatted_orders.sort_values("Start_time", inplace=True)

    # Initialize empty 'Start_time_date' column
    croom_reformatted_orders["Start_time_date"] = None

    def working_hours_shift(row):
        days = [d * 480 for d in range(1, 6)]

        for k, day in enumerate(days):
            if row["Start_time"] < day:
                row["Start_time_date"] = (
                    base_date
                    + pd.to_timedelta(row["Start_time"], unit="m")
                    + pd.Timedelta(days=k)
                    - pd.Timedelta(minutes=480 * k)
                )
                break
        return row

    # Apply function
    croom_reformatted_orders = croom_reformatted_orders.apply(working_hours_shift, axis=1)

    # Overwrite the integer start time with the calculated datetimes
    croom_reformatted_orders["Start_time"] = croom_reformatted_orders["Start_time_date"]
    croom_reformatted_orders["End_time"] = croom_reformatted_orders["Start_time"] + pd.to_timedelta(
        croom_reformatted_orders["duration"], unit="m"
    )

    # Reorder by earliest start time
    croom_reformatted_orders.sort_values(by="Start_time", inplace=True)

    # Check if the start time for each task within each job is later than the completion time of the previous task
    for job_id in croom_reformatted_orders["Job"].unique():
        job_schedule = croom_reformatted_orders[croom_reformatted_orders["Job"] == job_id]
        for i in range(1, len(job_schedule)):
            if not job_schedule.iloc[i]["Start_time"] >= job_schedule.iloc[i - 1]["End_time"]:
                logger.warning(
                    f"The start time for task {job_schedule.iloc[i]['task']} in job {job_id} "
                    f"is earlier than the completion time of the previous task!"
                )

    # Check if the start time for each task within each job is later than the completion time of the previous task
    for machine in croom_reformatted_orders["Machine"].unique():
        machine_schedule = croom_reformatted_orders[
            croom_reformatted_orders["Machine"] == machine
        ].sort_values("Start_time")
        for i in range(1, len(machine_schedule)):
            if not machine_schedule.iloc[i]["Start_time"] >= machine_schedule.iloc[i - 1]["End_time"]:
                logger.warning(
                    f"The start time for job {machine_schedule.iloc[i]['Job']}, task {machine_schedule.iloc[i]['task']}"
                    f" in machine {machine} is earlier than the completion time of the previous task!"
                )

    # Sort again by job and task before plotting
    croom_reformatted_orders = croom_reformatted_orders.sort_values(["Job", "task"])

    return croom_reformatted_orders


def create_chart(
    schedule: pd.DataFrame, parameters: Dict[str, Union[str, Dict[str, str]]]
) -> pd.DataFrame:
    """
    Creates a Gantt chart based on the schedule and parameters.

    Args:
        schedule (pd.DataFrame): The schedule data.
        parameters (Dict[str, Union[str, Dict[str, str]]]): The parameters for creating the chart.

    Returns:
        pd.DataFrame: The updated schedule data with additional columns for the chart.
    """
    if not is_string_dtype(schedule[[parameters["column_mapping"]["Resource"]]]):
        schedule[parameters["column_mapping"]["Resource"]] = schedule[
            parameters["column_mapping"]["Resource"]
        ].apply(str)
    schedule = schedule.rename(columns=parameters["column_mapping"])

    return schedule


def save_chart_to_html(gantt_chart: plotly.graph_objs.Figure) -> None:
    """
    Saves the Gantt chart to an HTML file.

    Args:
        gantt_chart (plotly.graph_objs.Figure): The Gantt chart to be saved.
    """
    filepath = Path(os.getcwd()) / "data/08_reporting/gantt_chart.html"
    plotly.offline.plot(gantt_chart, filename=str(filepath))

In [4]:
run(
        self,
        input_repr_dict: Dict[str, any],
        scheduling_options: dict,
        n: int = 1500,
        n_e: float = 0.1,
        n_c: float = 0.3,
        minutes_per_day: int = 480,
        max_iterations: int = 100,
    )

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│         input_repr_dict: Dict[str, any],                                                         │
│                        ▲                                                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax

In [9]:
run(self,input_repr_dict, 1500, 0.1, 0.3, 480, 100, )

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\omid\miniconda3\envs\EIT-Epsilon\Lib\site-packages\IPython\core\magics\execution.py:716 │
│ in run                                                                                           │
│                                                                                                  │
│ C:\Users\omid\miniconda3\envs\EIT-Epsilon\Lib\site-packages\IPython\utils\path.py:91 in          │
│ get_py_filename                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OSError: File `'(self,input_repr_dict,'` not found.

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 get_ipython().run_line_magic('run', '(self,input_repr_dict, 1500, 0.1, 0.3, 480, 100, )'     │
│   2                                                                                              │
│                                                                                                  │
│ C:\Users\omid\miniconda3\envs\EIT-Epsilon\Lib\site-packages\IPython\core\interactiveshell.py:248 │
│ 0 in run_line_magic                                                                              │
│                                                                                                  │
│ C:\Users\omid\miniconda3\envs\EIT-Epsilon\Lib\site-packages\IPython\core\magics\execution.py:727 │
│ in run                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
Exception: File `'(self,input_repr_dict,'` not found.